# People Analytics Case Study

## Libraries

In [1]:
# import libraries
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import getpass

## Exporting Data for Tableau Visualization

In [2]:
# Connection parameters
bd = "hr_data"
password = "password"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/' + bd

# Create the connection engine
engine = create_engine(connection_string)

In [3]:
# SQL query to execute
consulta = "SELECT * FROM hr_data.employees"

# Read the SQL query into a DataFrame
df_sql = pd.read_sql(consulta, con=engine)

# Save the DataFrame to a CSV file
df_sql.to_csv('../data/cleaned/hr_clean.csv', index=False)

# Display the first rows of the DataFrame
df_sql

,emp_id,first_name,last_name,birth_date,age,gender,race,department,job_title,location,hire_date,term_date,location_state
0,00-0037846,Kimmy,Walczynski,1991-06-04,33,Male,Hispanic or Latino,Engineering,Programmer Analyst I,Headquarters,2002-01-20,NaT,California
1,00-0041533,Ignatius,Springett,1984-06-29,40,Male,White,Business Development,Business Analyst,Headquarters,2019-04-08,NaT,California
2,00-0045747,Corbie,Bittlestone,1989-07-29,35,Male,Black or African American,Sales,Solutions Engineer Manager,Headquarters,2010-10-12,NaT,California
3,00-0055274,Baxy,Matton,1982-09-14,42,Female,White,Services,Service Tech,Headquarters,2005-04-10,NaT,California
4,00-0116166,Kacie,Offiler,1971-01-18,53,Male,Asian,Engineering,Developer III,Headquarters,2018-09-01,NaT,California
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20879,99-9797418,Dorella,Garvan,1998-07-08,26,Female,Hispanic or Latino,Research and Development,Research Assistant I,Headquarters,2012-02-08,NaT,California
20880,99-9869877,Dasie,Thorsby,2001-04-19,23,Female,Two or More Races,Services,Service Manager,Headquarters,2017-10-06,NaT,California
20881,99-9919822,Nerty,Wilding,1970-02-09,54,Female,Two or More Races,Training,Junior Trainer,Headquarters,2001-02-08,NaT,California
20882,99-9960380,Mabelle,Dawks,1985-09-02,39,Male,Two or More Races,Accounting,Staff Accountant I,Headquarters,2005-04-03,2012-12-10,California
